In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from src.gen import train_test_from_null
from src.titanic import prepare_dataset_pandas
from src.settings import DATA_PATH, DEVICE

# Whether to run intensive grid searches (True) or simple fits (False)
intensive = True

Let's use a previously cleaned database with missing values already imputed (including age)

In [ ]:
clean_comb_data = pd.read_csv(DATA_PATH / "titanic" / "all_data_clean.csv", index_col=0)

clean_train_data, clean_test_data = train_test_from_null(clean_comb_data, "Survived")
clean_train_data.info()
clean_test_data.info()

In [ ]:
class Net(nn.Module):
    def __init__(self, l1=128, l2=64):
        super(Net, self).__init__()
        self.hid1 = nn.Linear(10, l1)
        self.hid2 = nn.Linear(l1, l2)
        self.oupt = nn.Linear(l2, 2)

        nn.init.xavier_uniform_(self.hid1.weight)
        nn.init.zeros_(self.hid1.bias)
        nn.init.xavier_uniform_(self.hid2.weight)
        nn.init.zeros_(self.hid2.bias)
        nn.init.xavier_uniform_(self.oupt.weight)
        nn.init.zeros_(self.oupt.bias)

    def forward(self, x):
        x = torch.relu(self.hid1(x))
        x = torch.relu(self.hid2(x))
        x = self.oupt(x)  # no softmax: CrossEntropyLoss()
        return x

net = Net().to(DEVICE)

In [ ]:
train_data = clean_train_data.copy()
test_data = clean_test_data.copy()

target = "Survived"
drop_columns = ["PassengerId", "Cabin", "Ticket", "Name", "Sex"]

data = prepare_dataset_pandas([train_data, test_data], drop=drop_columns, target=target)
train_data = data[0]
test_data = data[1]
train_data

In [ ]:
train_target = torch.tensor(train_data[target].values).type(torch.LongTensor)
train = torch.tensor(train_data.drop(target, axis=1).values.astype(np.float32))

train_tensor = TensorDataset(train, train_target)
train_loader = DataLoader(dataset=train_tensor, batch_size=32, shuffle=True)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # print statistics
    print(f'{epoch + 1} loss: {running_loss / len(train_loader):.3f}')

print('Finished Training')